In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [2]:
csl_df = pd.read_csv('../../data/csl/Sites/CSL_he_PI_s49.csv', index_col=0)
X = csl_df.drop(['high_EBL','Sitenum'], axis=1, inplace=False)
y = csl_df['high_EBL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=7)
X_train                                                    

,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,...,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI
MomID,,,,,,,,,,,,,,,,,,,,,
49-06751,0,0,8,2,3,3.0,90,60.0,1,150.0,...,0,0,0,0,0,0,26,24,1,1
49-23935,0,0,8,8,8,3.0,65,0.0,77,114.0,...,0,0,0,0,1,0,39,24,1,1
49-06958,0,0,8,2,2,0.0,85,20.0,1,140.0,...,0,0,0,0,0,0,24,21,1,1
49-14728,0,0,8,8,8,2.0,72,0.0,77,115.0,...,0,0,0,0,0,0,34,24,1,1
49-24757,0,0,8,8,8,2.0,74,70.0,1,119.0,...,0,0,0,0,0,0,31,23,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49-10591,0,0,8,3,2,0.0,78,20.0,1,124.0,...,0,0,0,0,0,0,19,21,0,1
49-18110,0,0,8,2,3,1.0,77,90.0,1,115.0,...,0,0,0,0,0,0,18,26,0,1
49-00602,0,0,8,8,8,99.0,80,0.0,77,127.0,...,0,0,0,0,1,0,34,24,1,1


In [3]:
X.shape

(21322, 195)

<h1 align='center'>Exploratory Data Analysis</h1>

<h1 align='center'>Modelling</h1>

In [8]:
print(X_train.shape)
print(X.shape)
print(y.shape)

(14925, 195)
(21322, 195)
(21322,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [9]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [10]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(1608, 195)
(1608,)
[804 804]


<h1 align='center'>Basic Gradient Boosting</h1>

In [11]:
%%time
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
gb_rus.fit(X_rus, y_rus)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[4438 1614]
 [  32  313]]
              precision    recall  f1-score   support

           0       0.99      0.73      0.84      6052
           1       0.16      0.91      0.28       345

    accuracy                           0.74      6397
   macro avg       0.58      0.82      0.56      6397
weighted avg       0.95      0.74      0.81      6397

ROC AUC prob1: 0.8688599768192573
MCC: 0.3153716353279921
F2: 0.4732385848200787
PR_AUC: 0.2205171513715284
CPU times: user 608 ms, sys: 5.86 ms, total:

In [12]:
%%time
# Not undersampled
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_weight = GradientBoostingClassifier()  
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
print(gb_weight.get_params())
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[4490 1562]
 [  38  307]]
              precision    recall  f1-score   support

           0       0.99      0.74      0.85      6052
           1       0.16      0.89      0.28       345

    accuracy                           0.75      6397
   macro avg       0.58      0.82      0.56      6397
weighted avg       0.95      0.75      0.82      6397

ROC AUC prob1: 0.8727331724091688
MCC: 0.3137995331532645
F2: 0.4724530624807633
PR_AUC: 0.2435054511986828
CPU times: user 4.99 s, sys: 6.66 ms, total:

<h1 align='center'>Hyperparameter Tuning with Cross Validation</h1>

In [13]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    max_depth = trial.suggest_int('max_depth',3,20)
    max_features = trial.suggest_int('max_features',10,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.08,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate) 
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train_c, X_test_c = X.iloc[train_idx], X.iloc[test_idx]
        y_train_c, y_test_c = y[train_idx], y[test_idx]
        sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
        X_res, y_res = sampler.fit_resample(X_train_c, y_train_c)

#        weights = class_weight.compute_sample_weight('balanced', y=y_train)
#        model.fit(X_train, y_train, sample_weight=weights)
        model.fit(X_res, y_res)
        y_pred = model.predict(X_test_c)
        probs = model.predict_proba(X_test_c)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test_c, prob1)
        roc = roc_auc_score(y_test_c, prob1)
        roc_list.append(roc)
        mcc = matthews_corrcoef(y_test_c, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test_c, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        pr_list.append(pr_auc)
        avg_precision = average_precision_score(y_test_c, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
        
    print(f'MCC: {mcc_list}')
    print(f'ROC: {roc_list}')
    print(f'PR_AUC: {pr_list}')
    return np.mean(mcc_list)

sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=25)
#study.optimize(objective,n_trials=5)

[I 2021-12-10 14:30:13,218] A new study created in memory with name: no-name-0150141f-e408-4915-be77-9b99a2492e26
[I 2021-12-10 14:30:15,013] Trial 0 finished with value: 0.32435021248539964 and parameters: {'n_estimators': 24, 'subsample': 0.955983758448023, 'max_depth': 10, 'max_features': 17, 'learning_rate': 0.1973587414395923, 'min_samples_leaf': 27}. Best is trial 0 with value: 0.32435021248539964.


MCC: [0.33864621685917395, 0.3258690257197002, 0.3120587371594714, 0.32082687020325285]
ROC: [0.8842147291983852, 0.8750034426157269, 0.8618093455516012, 0.873624805764502]
PR_AUC: [0.23255702238537396, 0.23414921130400373, 0.2013871898174007, 0.20961815662203806]


[I 2021-12-10 14:30:19,426] Trial 1 finished with value: 0.32110738936992844 and parameters: {'n_estimators': 105, 'subsample': 0.8144102266719524, 'max_depth': 7, 'max_features': 15, 'learning_rate': 0.16150759953451288, 'min_samples_leaf': 41}. Best is trial 0 with value: 0.32435021248539964.


MCC: [0.33526767522541384, 0.31484822036817267, 0.3154734966375594, 0.3188401652485679]
ROC: [0.8771065494726545, 0.8642652356401612, 0.8589302728244415, 0.873085195541341]
PR_AUC: [0.21851472435568095, 0.2266558526169103, 0.19820987117942546, 0.2243810758754444]


[I 2021-12-10 14:30:23,526] Trial 2 finished with value: 0.32838807160890915 and parameters: {'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}. Best is trial 2 with value: 0.32838807160890915.


...

MCC: [0.35009721322138015, 0.31885288333368256, 0.3237236569345528, 0.3238972922651934]
ROC: [0.884441997529752, 0.8801683989909006, 0.874806282693574, 0.8676490198232483]
PR_AUC: [0.24192298006630064, 0.2615143634276929, 0.2374655434074323, 0.21861942366341264]


[I 2021-12-10 14:32:35,707] Trial 23 finished with value: 0.3272808914583817 and parameters: {'n_estimators': 121, 'subsample': 0.8847667467206112, 'max_depth': 20, 'max_features': 19, 'learning_rate': 0.0942433137579249, 'min_samples_leaf': 9}. Best is trial 12 with value: 0.33390160754165166.


MCC: [0.34538176306802654, 0.3179507077250074, 0.32035458881926165, 0.3254365062212313]
ROC: [0.8840482499647699, 0.8705955174389143, 0.8651969370037884, 0.8703090702191121]
PR_AUC: [0.22947330901048357, 0.2122693186996103, 0.20682132583730034, 0.21765794264793922]


[I 2021-12-10 14:32:39,248] Trial 24 finished with value: 0.32817972028124953 and parameters: {'n_estimators': 101, 'subsample': 0.8551963642689071, 'max_depth': 5, 'max_features': 20, 'learning_rate': 0.0809593886171461, 'min_samples_leaf': 20}. Best is trial 12 with value: 0.33390160754165166.


MCC: [0.3418141229028901, 0.3235876163049037, 0.31971090455694817, 0.3276062373602563]
ROC: [0.8808146844355041, 0.8829916881485889, 0.8727756624043678, 0.8790126169299426]
PR_AUC: [0.22639883440527078, 0.2702863426642678, 0.21640790789180112, 0.24068260403748162]
CPU times: user 2min 25s, sys: 380 ms, total: 2min 26s
Wall time: 2min 26s


In [14]:
best = study.best_trial
best

FrozenTrial(number=12, values=[0.33390160754165166], datetime_start=datetime.datetime(2021, 12, 10, 14, 31, 22, 259735), datetime_complete=datetime.datetime(2021, 12, 10, 14, 31, 25, 554835), params={'n_estimators': 129, 'subsample': 0.845528398647246, 'max_depth': 3, 'max_features': 20, 'learning_rate': 0.08006789436397342, 'min_samples_leaf': 10}, distributions={'n_estimators': IntUniformDistribution(high=200, low=10, step=1), 'subsample': UniformDistribution(high=1.0, low=0.8), 'max_depth': IntUniformDistribution(high=20, low=3, step=1), 'max_features': IntUniformDistribution(high=20, low=10, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.08), 'min_samples_leaf': IntUniformDistribution(high=50, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=12, state=TrialState.COMPLETE, value=None)

In [15]:
%%time
gb_params = study.best_params
#gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[4477 1575]
 [  33  312]]
              precision    recall  f1-score   support

           0       0.99      0.74      0.85      6052
           1       0.17      0.90      0.28       345

    accuracy                           0.75      6397
   macro avg       0.58      0.82      0.56      6397
weighted avg       0.95      0.75      0.82      6397

ROC AUC prob1: 0.8728847572248245
MCC: 0.31903644411605014
F2: 0.47750229568411384
PR_AUC: 0.23795067997920405
CPU times: user 233 ms, sys: 2.01 ms, total: 235 ms
Wall time: 232 ms


In [16]:
%%time
gb_params = study.best_params
gb_params['random_state'] = 42
gb_weight = GradientBoostingClassifier(**gb_params)
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[4441 1611]
 [  33  312]]
              precision    recall  f1-score   support

           0       0.99      0.73      0.84      6052
           1       0.16      0.90      0.28       345

    accuracy                           0.74      6397
   macro avg       0.58      0.82      0.56      6397
weighted avg       0.95      0.74      0.81      6397

ROC AUC prob1: 0.8703842064427137
MCC: 0.31437459088501823
F2: 0.47229791099000906
PR_AUC: 0.2256715279314253
CPU times: user 1.1 s, sys: 3.98 ms, total: 1.1 s
Wall time: 1.1 s


In [17]:
# No Sitenum - 5
coeffs_wgt = pd.Series(data=gb_weight.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs_wgt[0:20]

Delmode         0.358040
Hxanemia        0.095787
Delfetalpos     0.079318
Lac_Min         0.053093
Admefface       0.052234
TrialLabor      0.043442
Lac_None        0.033449
transfus_yes    0.019314
uscar           0.017466
CS_FTP          0.015495
BESTGA          0.012806
Intrafever      0.012616
Admcervpos      0.012163
CS_UScar        0.012133
Admpresent      0.011141
new_age         0.011053
CS_NRFHT        0.008042
new_BMI         0.006668
AdmSBP          0.006603
AdmDBP          0.006562
dtype: float64

In [1]:
plt.rcParams["figure.figsize"] = (14, 8)
x_labels = coeffs_wgt.index.values[0:25]
print(x_labels)
fig, ax = plt.subplots(1,1)  # Create a figure and an axes.
ax.set_title('Top Predictors: All Sites combined; target: High_EBLoss, vars: PI')
ax.bar(x_labels, coeffs_wgt.values[0:25])
ax.set_ylabel('Coeff')
ax.set_xlabel('Variable')
plt.draw()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=16)
#plt.xlim(1,20)
plt.show()

NameError: name 'plt' is not defined

In [19]:
# No Sitenum - 5
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs[0:20]

Delmode         0.296163
Lac_None        0.082153
Hxanemia        0.074459
Lac_Min         0.067637
CS_FTP          0.058739
Admpresent      0.046754
Delfetalpos     0.036141
Intrafever      0.025021
Admefface       0.024864
new_age         0.023217
AdmDBP          0.019894
BESTGA          0.018348
uscar           0.018166
Dilat_lst       0.017962
prelaborCD      0.017912
transfus_yes    0.014563
AdmSBP          0.011502
Anteanemia      0.009878
CS_NRFHT        0.009177
Admreason       0.007574
dtype: float64

In [21]:
gb_params # 5 runs

{'n_estimators': 129,
 'subsample': 0.845528398647246,
 'max_depth': 3,
 'max_features': 20,
 'learning_rate': 0.08006789436397342,
 'min_samples_leaf': 10,
 'random_state': 42}

In [2]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [ ]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

In [24]:
np.mean([0.3149689279176914, 0.31449457458602254, 0.31066798538596907, 0.3219519143488398])

0.3155208505596307

<h1 align='center'>Thanks and make sure to learn!</h1>